In [59]:
INPUT_CSV = "G:\Real Estate Market Analysis\Model_3\Model3_SellingPrices_combined2.csv"
OUTPUT_CSV = "G:\Real Estate Market Analysis\Model_4\Model4_Buy_NoBuy2.csv"


<>:1: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:1: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
C:\Users\dheer\AppData\Local\Temp\ipykernel_10500\1866662652.py:1: SyntaxWarning: invalid escape sequence '\R'
  INPUT_CSV = "G:\Real Estate Market Analysis\Model_3\Model3_SellingPrices_combined2.csv"
C:\Users\dheer\AppData\Local\Temp\ipykernel_10500\1866662652.py:2: SyntaxWarning: invalid escape sequence '\R'
  OUTPUT_CSV = "G:\Real Estate Market Analysis\Model_4\Model4_Buy_NoBuy2.csv"


In [60]:
import numpy as np
import pandas as pd


In [61]:
# -----------------------------
# Model 4 v3.1 - Calibrated Policy Knobs
# -----------------------------

# Transaction costs (slightly softened but realistic)
BUY_CLOSING_COST_PCT  = 0.02    # 2%
SELL_CLOSING_COST_PCT = 0.048   # 4.8%

# Holding cost model
# Monthly holding cost as % of acquisition price
HOLDING_COST_PCT_OF_BUY_PER_MONTH = 0.0018  # 0.18% per month

# Required debt hurdle
DEBT_HURDLE_RATE = 0.09

# Base margin decision thresholds (tiered)
# < 7.5% -> NO_BUY
# 7.5% - 8.5% -> REVIEW_REQUIRED
# >= 8.5% -> eligible BUY (subject to other checks + hurdle)
MIN_MARGIN_BASE_BUY = 0.085
MIN_MARGIN_BASE_REVIEW = 0.075

# Bear margin guard (allow small downside in MVP)
MIN_MARGIN_BEAR = -0.02

# Liquidity proxy threshold
LIQUIDITY_OK_THRESHOLD = 0.45

# Stop-loss buffer on bear sell price
STOP_LOSS_BUFFER_PCT = 0.02

# Reno sanity check
# If reno cost > 35% of acquisition price -> force REVIEW
RENO_TO_BUY_REVIEW_THRESHOLD = 0.35


# -----------------------------
# Horizon analysis settings
# -----------------------------

HORIZON_MONTHS = {
    "q4_4m": 4,
    "m6": 6,
    "y1": 12,
    "y2": 24,
    "y3": 36
}

# Appreciation assumptions for hold sensitivity
ANNUAL_APPRECIATION = {
    "base": 0.03,  # 3% per year
    "bull": 0.06,  # 6%
    "bear": 0.00   # 0%
}


In [62]:
def safe_col(df, col, default=np.nan):
    if col in df.columns:
        return df[col]
    return pd.Series([default] * len(df))


def future_price(current_price, months, annual_rate):
    """
    Compound appreciation:
    future = current * (1 + rate)^(months/12)
    """
    return current_price * ((1 + annual_rate) ** (months / 12.0))


In [63]:
def compute_model4_v3_1(df):
    df = df.copy()

    # -----------------------------
    # Inputs from Model 3 (+ Model 2 carryover)
    # -----------------------------
    predicted_fair_price = safe_col(df, "predicted_fair_price", default=np.nan).astype(float)
    max_offer_price = safe_col(df, "max_offer_price", default=np.nan).astype(float)

    # Fallback if max_offer missing
    max_offer_price = max_offer_price.fillna(predicted_fair_price * 0.95)

    estimated_reno_cost = safe_col(df, "estimated_reno_cost", default=0).fillna(0).astype(float)

    sell_base = safe_col(df, "predicted_sell_price_base", default=np.nan).astype(float)
    sell_bull = safe_col(df, "predicted_sell_price_bull", default=np.nan).astype(float)
    sell_bear = safe_col(df, "predicted_sell_price_bear", default=np.nan).astype(float)

    # Derive bull/bear if missing
    sell_bull = sell_bull.fillna(sell_base * 1.04)
    sell_bear = sell_bear.fillna(sell_base * 0.96)

    # Inherited Model 2 uncertainty flag (if present)
    review_flag = safe_col(df, "review_flag", default="OK").fillna("OK").astype(str).str.upper()

    # -----------------------------
    # Flip holding months heuristic
    # -----------------------------
    reno_tier = safe_col(df, "reno_tier", default="Medium").astype(str)
    holding_months_map = {"Light": 4, "Medium": 6, "Heavy": 9}
    expected_holding_months = reno_tier.map(holding_months_map).fillna(6).astype(int)

    # -----------------------------
    # Cost stack (flip baseline)
    # -----------------------------
    buy_closing_cost = max_offer_price * BUY_CLOSING_COST_PCT
    holding_cost_monthly = max_offer_price * HOLDING_COST_PCT_OF_BUY_PER_MONTH
    holding_cost_base = expected_holding_months * holding_cost_monthly
    sell_closing_cost_base = sell_base * SELL_CLOSING_COST_PCT

    total_cost = (
        max_offer_price
        + estimated_reno_cost
        + buy_closing_cost
        + holding_cost_base
        + sell_closing_cost_base
    )

    # Attach cost columns
    df["max_offer_price_used"] = max_offer_price
    df["estimated_reno_cost_used"] = estimated_reno_cost
    df["expected_holding_months"] = expected_holding_months
    df["buy_closing_cost"] = buy_closing_cost
    df["holding_cost_monthly"] = holding_cost_monthly
    df["holding_cost_base"] = holding_cost_base
    df["sell_closing_cost_base"] = sell_closing_cost_base
    df["total_cost"] = total_cost

    # Attach sell columns (flip)
    df["sell_base_used"] = sell_base
    df["sell_bull_used"] = sell_bull
    df["sell_bear_used"] = sell_bear

    # -----------------------------
    # Flip profits/margins
    # -----------------------------
    df["profit_base"] = sell_base - total_cost
    df["profit_bull"] = sell_bull - total_cost
    df["profit_bear"] = sell_bear - total_cost

    df["margin_pct_base"] = df["profit_base"] / total_cost
    df["margin_pct_bear"] = df["profit_bear"] / total_cost

    # 9% hurdle check
    df["meets_9pct_hurdle_flag"] = df["margin_pct_base"] >= DEBT_HURDLE_RATE

    # Liquidity proxy
    final_city_score = safe_col(df, "Final_City_Score", default=np.nan).astype(float)
    df["liquidity_flag"] = np.where(final_city_score.fillna(0) >= LIQUIDITY_OK_THRESHOLD, "OK", "REVIEW")

    # Stop-loss price
    df["stop_loss_price"] = sell_bear * (1 - STOP_LOSS_BUFFER_PCT)

    # Reno sanity ratio
    df["reno_to_buy_ratio"] = np.where(max_offer_price > 0, estimated_reno_cost / max_offer_price, np.nan)
    df["reno_risk_flag"] = np.where(df["reno_to_buy_ratio"] > RENO_TO_BUY_REVIEW_THRESHOLD, "REVIEW", "OK")

    # -----------------------------
    # Multi-horizon profitability
    # -----------------------------
    for key, months in HORIZON_MONTHS.items():
        # Future sell prices based on appreciation assumptions
        sell_base_h = future_price(sell_base, months, ANNUAL_APPRECIATION["base"])
        sell_bull_h = future_price(sell_base, months, ANNUAL_APPRECIATION["bull"])
        sell_bear_h = future_price(sell_base, months, ANNUAL_APPRECIATION["bear"])

        # Horizon-specific sell closing
        sell_close_h = sell_base_h * SELL_CLOSING_COST_PCT

        # Horizon-specific holding cost
        holding_cost_h = months * holding_cost_monthly

        # Total horizon cost
        total_cost_h = (
            max_offer_price
            + estimated_reno_cost
            + buy_closing_cost
            + holding_cost_h
            + sell_close_h
        )

        # Save columns
        df[f"sell_base_{key}"] = sell_base_h
        df[f"total_cost_{key}"] = total_cost_h

        df[f"profit_base_{key}"] = sell_base_h - total_cost_h
        df[f"profit_bull_{key}"] = sell_bull_h - total_cost_h
        df[f"profit_bear_{key}"] = sell_bear_h - total_cost_h

        df[f"margin_base_{key}"] = df[f"profit_base_{key}"] / total_cost_h

    # Convenience horizon feasibility flags
    df["buy_if_6m"] = df["margin_base_m6"] >= 0.09
    df["buy_if_y1"] = df["margin_base_y1"] >= 0.09

    # -----------------------------
    # Decision Logic (v3.1)
    # -----------------------------
    df["buy_decision"] = "BUY"
    df["decision_reason"] = "PASS"

    # Base margin tiering
    mask_base_nobuy = df["margin_pct_base"] < MIN_MARGIN_BASE_REVIEW
    df.loc[mask_base_nobuy, ["buy_decision", "decision_reason"]] = ["NO_BUY", "BASE_MARGIN_FAIL"]

    mask_base_review = (
        (df["buy_decision"] == "BUY") &
        (df["margin_pct_base"] >= MIN_MARGIN_BASE_REVIEW) &
        (df["margin_pct_base"] < MIN_MARGIN_BASE_BUY)
    )
    df.loc[mask_base_review, ["buy_decision", "decision_reason"]] = ["REVIEW_REQUIRED", "BASE_MARGIN_BORDERLINE"]

    # Bear margin weak
    mask_bear_weak = (df["buy_decision"] == "BUY") & (df["margin_pct_bear"] < MIN_MARGIN_BEAR)
    df.loc[mask_bear_weak, ["buy_decision", "decision_reason"]] = ["REVIEW_REQUIRED", "BEAR_MARGIN_WEAK"]

    # Liquidity
    mask_liq = (df["buy_decision"] == "BUY") & (df["liquidity_flag"] == "REVIEW")
    df.loc[mask_liq, ["buy_decision", "decision_reason"]] = ["REVIEW_REQUIRED", "LIQUIDITY_RISK"]

    # Reno risk
    mask_reno = (df["buy_decision"] == "BUY") & (df["reno_risk_flag"] == "REVIEW")
    df.loc[mask_reno, ["buy_decision", "decision_reason"]] = ["REVIEW_REQUIRED", "RENO_COST_RISK"]

    # Model 2 uncertainty
    mask_m2 = (df["buy_decision"] == "BUY") & (review_flag == "REVIEW_REQUIRED")
    df.loc[mask_m2, ["buy_decision", "decision_reason"]] = ["REVIEW_REQUIRED", "M2_UNCERTAINTY"]

    # Final hurdle hard fail (only if still BUY)
    mask_hurdle_fail = (df["buy_decision"] == "BUY") & (~df["meets_9pct_hurdle_flag"])
    df.loc[mask_hurdle_fail, ["buy_decision", "decision_reason"]] = ["NO_BUY", "HURDLE_FAIL"]

    # -----------------------------
    # Optional: BUY_CONDITIONALLY
    # -----------------------------
    df["buy_decision_v2"] = df["buy_decision"]

    mask_conditional = (
        (df["buy_decision"] == "REVIEW_REQUIRED") &
        (df["buy_if_6m"] == True)
    )
    df.loc[mask_conditional, "buy_decision_v2"] = "BUY_CONDITIONALLY"

    return df


In [64]:
df3 = pd.read_csv(INPUT_CSV)
print("Model 3 shape:", df3.shape)
df3.head()


Model 3 shape: (1562, 34)


,property_id,metro,city,state,zip,beds,baths,sqft,lot_size,age,...,fast_close_flag,eco_upgrade_flag,reno_uplift_pct,eco_uplift_pct,cx_uplift_pct,total_uplift_pct,predicted_sell_price_base,predicted_sell_price_bull,predicted_sell_price_bear,reno_to_fair_ratio
0,1,Peoria,Peoria,IL,61615,3,1.0,1897,9170,56,...,1,1,0.222850,0.017138,0.020960,0.260948,235663.544904,245090.086700,226237.003108,0.2
1,2,Peoria,Morton,IL,61550,4,2.5,2168,13200,50,...,1,1,0.135441,0.021327,0.021319,0.178087,312187.549508,324675.051488,299700.047527,0.2
2,3,Peoria,Washington,IL,61571,3,1.0,874,11600,77,...,1,1,0.265009,0.031414,0.026541,0.322964,150128.477328,156133.616421,144123.338235,0.2
3,4,Peoria,Pekin,IL,61554,3,2.5,1928,17859,63,...,1,1,0.215465,0.028215,0.017551,0.261231,284733.395803,296122.731636,273344.059971,0.2
4,5,Peoria,Peoria Heights,IL,61616,2,1.0,720,6098,75,...,1,1,0.282668,0.021791,0.021618,0.326077,109133.555980,113498.898219,104768.213740,0.2


In [65]:
df4 = compute_model4_v3_1(df3)

print("Model 4 v3.1 shape:", df4.shape)
df4["buy_decision"].value_counts(dropna=False)


Model 4 v3.1 shape: (1562, 90)


buy_decision
NO_BUY             1360
REVIEW_REQUIRED     157
BUY                  45
Name: count, dtype: int64

In [66]:
df4["buy_decision_v2"].value_counts(dropna=False)


buy_decision_v2
NO_BUY               1360
BUY_CONDITIONALLY     157
BUY                    45
Name: count, dtype: int64

In [67]:
base_cols = [
    "property_id",
    "max_offer_price_used",
    "estimated_reno_cost_used",
    "reno_to_buy_ratio",
    "expected_holding_months",
    "total_cost",
    "sell_base_used",
    "sell_bear_used",
    "profit_base",
    "profit_bear",
    "margin_pct_base",
    "margin_pct_bear",
    "meets_9pct_hurdle_flag",
    "liquidity_flag",
    "reno_risk_flag",
    "stop_loss_price",
    "buy_if_6m",
    "buy_if_y1",
    "buy_decision",
    "buy_decision_v2",
    "decision_reason",
]

for key in HORIZON_MONTHS.keys():
    base_cols += [f"profit_base_{key}", f"margin_base_{key}"]

present = [c for c in base_cols if c in df4.columns]
df4[present].head(15)


,property_id,max_offer_price_used,estimated_reno_cost_used,reno_to_buy_ratio,expected_holding_months,total_cost,sell_base_used,sell_bear_used,profit_base,profit_bear,...,profit_base_q4_4m,margin_base_q4_4m,profit_base_m6,margin_base_m6,profit_base_y1,margin_base_y1,profit_base_y2,margin_base_y2,profit_base_y3,margin_base_y3
0,1,177549.20500,37378.780000,0.210526,9,232667.116376,235663.544904,226237.003108,2996.428527,-6430.113269,...,6815.819752,0.029483,7295.601731,0.031463,8768.213663,0.037477,11865.618203,0.049824,15170.996763,0.062601
1,2,251745.61370,52999.076560,0.210526,6,327483.457538,312187.549508,299700.047527,-15295.908031,-27783.410011,...,-11446.832961,-0.035035,-10870.812792,-0.033172,-9098.684245,-0.027517,-5352.830794,-0.015905,-1331.470583,-0.003888
2,3,107804.95180,22695.779320,0.210526,9,141609.437287,150128.477328,144123.338235,8519.040041,2513.900948,...,10904.449023,0.077496,11229.179429,0.079564,12224.562614,0.085842,14312.275047,0.098733,16532.476462,0.112072
3,4,214470.33440,45151.649360,0.210526,9,281053.012864,284733.395803,273344.059971,3680.382940,-7708.952893,...,8294.614194,0.029702,8874.469714,0.031683,10654.228918,0.037698,14397.615053,0.050048,18392.279548,0.062827
4,5,78183.17529,16459.615850,0.210526,9,102711.432773,109133.555980,104768.213740,6422.123207,2056.780968,...,8154.503493,0.079899,8391.223433,0.081972,9116.788419,0.088263,10638.391823,0.101182,12256.306026,0.114548
5,6,341559.14600,46806.841206,0.137039,4,417215.481139,407480.940862,391181.703228,-9734.540277,-26033.777911,...,-5893.480022,-0.014119,-5188.323979,-0.012390,-3015.336308,-0.007133,1593.771480,0.003700,6562.482827,0.014959
6,7,125957.60760,26517.391080,0.210526,9,165345.272164,173137.668514,166212.161773,7792.396350,866.889610,...,10558.072052,0.064263,10926.703331,0.066307,12057.037082,0.072513,14429.508925,0.085254,16954.775453,0.098438
7,8,234975.16510,49468.455800,0.210526,9,309175.172157,338030.214167,324509.005601,28855.042011,15333.833444,...,34156.211504,0.111178,34915.309397,0.113307,37240.319036,0.119770,42108.622674,0.133041,47275.239328,0.146772
8,9,279724.13850,38970.114817,0.139316,6,343905.369201,345741.925395,331912.248379,1836.556193,-11993.120823,...,6102.649432,0.017789,6737.267162,0.019576,8689.924887,0.025012,12818.504566,0.036213,17252.202878,0.047850
9,10,270753.23510,46637.233866,0.172250,6,341708.001420,332881.933602,319566.656258,-8826.067818,-22141.345162,...,-4713.492687,-0.013827,-4107.640507,-0.012013,-2243.094733,-0.006500,1700.956653,0.004840,5938.777677,0.016598


In [68]:
df4.to_csv(OUTPUT_CSV, index=False)
print("Saved:", OUTPUT_CSV)


Saved: G:\Real Estate Market Analysis\Model_4\Model4_Buy_NoBuy2.csv


# Model 4 – Buy/No-Buy Decision Model

**Risk-Adjusted Investment Underwriting with Margin, Liquidity, Stop-Loss & Multi-Horizon Profitability**

---

## Overview

Model 4 is the final underwriting layer that determines whether a property should be acquired. The model combines acquisition price discipline from **Model 2** and ARV + renovation economics from **Model 3** to compute expected profitability under structured business rules.

Model 4 operates as an **AI + policy-based investment decision engine**.

The model produces:

* **Total Project Cost**
* **Base/Bull/Bear Profit + Margin**
* **Liquidity Check**
* **Reno Risk Flag**
* **Stop-Loss Price**
* **Buy/No-Buy/Review Decision**
* **Multi-Horizon Profit Projections**

  * **Q4 (4 months)**
  * **6 months**
  * **Year 1**
  * **Year 2**
  * **Year 3**

---

## Key Objectives

* Decide acquisition eligibility using consistent rules
* Ensure profitability is aligned to the **≥9% debt-financing hurdle**
* Incorporate **stop-loss and liquidity screening**
* Route uncertain cases to analysts
* Provide horizon-based profit sensitivity for strategy planning

---

## Data Sources Used

### 1. Model-2 Outputs

* `max_offer_price`
* `predicted_fair_price`
* `review_flag`
* `Final_City_Score`

### 2. Model-3 Outputs

* `estimated_reno_cost`
* `reno_tier`
* `predicted_sell_price_base`
* `predicted_sell_price_bull`
* `predicted_sell_price_bear`

### 3. Business & Financial Assumptions (MVP)

* buy closing cost %
* sell closing cost %
* monthly holding cost %
* tier-based holding duration
* base/bull/bear appreciation assumptions for longer-hold horizons

---

## Feature Engineering Pipeline

Model-4 builds investment economics using:

### Core Cost Stack

```
total_cost
= max_offer_price
+ estimated_reno_cost
+ buy_closing_cost
+ holding_cost
+ sell_closing_cost
```

Where:

* `holding_cost = holding_cost_monthly × expected_holding_months`

### Base Profit & Margin

```
profit_base = sell_base - total_cost
margin_base = profit_base / total_cost
```

---

## Risk Controls

### 1. Liquidity Proxy

Model-4 uses a market-quality proxy:

* `Final_City_Score`

Decision use:

* Score ≥ threshold → `OK`
* Else → `REVIEW`

This avoids heavy data dependency at MVP stage while preserving market-awareness.

### 2. Renovation Risk Flag

A cost sanity ratio protects against over-capitalized rehabs:

```
reno_to_buy_ratio = estimated_reno_cost / max_offer_price
```

If this ratio exceeds a threshold:

* validation is required

### 3. Stop-Loss Price

A downside exit floor is computed using bear-case ARV:

```
stop_loss_price = sell_bear × (1 - buffer)
```

This formalizes capital protection rules for high-risk markets.

---

## Multi-Horizon Profitability

To support strategic flexibility, Model-4 computes base-case profit and margin under:

* `Q4 (4 months)`
* `6 months`
* `Year 1`
* `Year 2`
* `Year 3`

This is calculated using appreciation-based sensitivity assumptions applied to `sell_base`.

These outputs enable:

* short-term flip viability
* medium-hold fallback strategies
* longer-hold risk awareness

---

## Decision Logic

Model-4 uses a tiered rule-based gate:

* If base margin is below a lower threshold → `NO_BUY`
* If base margin is borderline → `REVIEW_REQUIRED`
* If base margin meets the buy threshold and passes:

  * liquidity check
  * reno risk check
  * Model-2 uncertainty check
  * ≥9% hurdle requirement
    → `BUY`

This creates a disciplined AI + policy underwriting workflow.

---

## Model-4 Output Columns

| Column                                      | Description                      |
| ------------------------------------------- | -------------------------------- |
| `max_offer_price_used`                      | Final acquisition ceiling used   |
| `estimated_reno_cost_used`                  | Final renovation budget used     |
| `expected_holding_months`                   | Tier-based hold estimate         |
| `total_cost`                                | Full project cost                |
| `sell_base_used`                            | Base ARV                         |
| `sell_bear_used`                            | Downside ARV                     |
| `profit_base`, `profit_bull`, `profit_bear` | Flip profitability               |
| `margin_pct_base`, `margin_pct_bear`        | Flip margins                     |
| `meets_9pct_hurdle_flag`                    | Financing-aligned hurdle check   |
| `liquidity_flag`                            | Market liquidity proxy           |
| `reno_to_buy_ratio`                         | Reno sanity ratio                |
| `reno_risk_flag`                            | Reno risk routing                |
| `stop_loss_price`                           | Risk control exit floor          |
| `buy_decision`                              | `BUY / NO_BUY / REVIEW_REQUIRED` |
| `decision_reason`                           | Explains rule trigger            |
| `profit_base_q4_4m`                         | Q4 horizon profit                |
| `profit_base_m6`                            | 6-month horizon profit           |
| `profit_base_y1`                            | 1-year horizon profit            |
| `profit_base_y2`                            | 2-year horizon profit            |
| `profit_base_y3`                            | 3-year horizon profit            |
| `margin_base_*`                             | Horizon margins                  |

---

## Repository Structure (Model-4 Folder)

```
Model_4/
│
├── Model4_BuyNoBuy.ipynb
├── Model4_Buy_NoBuy.csv
└── inputs/
    ├── Model2_AcquisitionPrices_combined.csv
    └── Model3_SellingPrices_combined.csv
```

---

## How to Run Model-4

1. Ensure Model-3 output exists:

   Model3_SellingPrices_combined.csv

2. Open notebook:

   Model_4/Model4_BuyNoBuy.ipynb

3. Run all cells.

The notebook will:

* Load Model-3 outputs
* Build cost stack
* Compute Base/Bull/Bear profits
* Apply liquidity + reno risk checks
* Enforce ≥9% hurdle logic
* Generate multi-horizon profitability
* Export final decision CSV

---

## Example Output (Sample Row)

```
property_id: 10392
max_offer_price_used: 177,548
estimated_reno_cost_used: 32,500
total_cost: 222,100
sell_base_used: 246,000
profit_base: 23,900
margin_pct_base: 0.108
liquidity_flag: OK
buy_decision: BUY
profit_base_m6: 25,400
profit_base_y1: 29,800
```

---

## Why Model-4 Is Critical

Model-4 establishes:

* A disciplined final acquisition gate
* Transparent profitability and risk logic
* A structured AI + human review loop
* A portfolio-ready output usable for:

  * investment memos
  * underwriting dashboards
  * pro forma financial statements
  * scenario-driven expansion planning

---

## Conclusion

Model-4 converts acquisition price and ARV predictions into a structured, risk-adjusted investment decision. By combining cost discipline, scenario margins, liquidity and stop-loss controls, and multi-horizon profitability, it completes the full AI-driven real estate investment pipeline.

